In [13]:
text="""Bonjour! 🌟 Welcome to the magical world of language! In this vast universe of words, where the sky is the limit, let's embark on a journey together, exploring the beauty and diversity of different languages. ¡Hola amigos! ¿Cómo están? Today, we're going to delve into the wonders of multilingualism. Imagine being able to speak fluently in various tongues, connecting with people from all walks of life. C'est fantastique! 🎉

Let's start with English, the language of Shakespeare and modern communication. English is a global language, spoken by millions around the world. From the bustling streets of New York City to the serene countryside of England, English bridges cultures and societies. It's a language of opportunity, innovation, and creativity. So, grab your cup of tea ☕️ and let's dive into the world of English literature and culture.

Pero no podemos olvidar el hermoso idioma español. Con sus ricos sonidos y expresiones poéticas, el español nos lleva en un viaje a través de la pasión y el romance. Desde las vibrantes fiestas de España hasta las playas doradas de América Latina, el español es una celebración de la vida y la diversidad. ¡Viva la lengua española! 🇪🇸

Maintenant, parlons français! Ah, la langue de l'amour et de la sophistication. Le français est un mélange envoûtant de finesse et d'élégance. De Paris, la ville lumière, aux champs de lavande de la Provence, le français évoque un sentiment de joie de vivre. C'est magnifique! 🥖

As we journey through these languages, let's not forget the power of emojis. 😊 Emojis add color and emotion to our digital conversations. Whether it's a smiley face 😄 to brighten someone's day or a heart ❤️ to express love and affection, emojis transcend language barriers and connect us on a deeper level.

Now, let's wrap up our linguistic adventure with a toast 🥂 to the beauty of language! May we continue to explore, learn, and appreciate the rich tapestry of words that make our world a more vibrant and interconnected place. Cheers to language! Salud! 🎊

"""

In [65]:
text="""

Main menu

WikipediaThe Free Encyclopedia
Search Wikipedia
Search
Create account
Log in

Personal tools
Contents hide
(Top)
Characteristics
Toggle Characteristics subsection
History
Alternative definitions
Implementations
Trust and security
Toggle Trust and security subsection
Communities
Toggle Communities subsection
Conferences
Legal environment
See also
Notes
References
Toggle References subsection
Further reading
External links
Wiki

Article
Talk
Read
View source
View history

Tools
Page semi-protected
Listen to this article
From Wikipedia, the free encyclopedia
For other uses, see Wiki (disambiguation).
refer to caption
Editing display showing MediaWiki markup language
A wiki (/ˈwɪki/ ⓘ WI-kee) is a form of online hypertext publication that is collaboratively edited and managed by its own audience directly through a web browser. A typical wiki contains multiple pages for the subjects or scope of the project, and could be either open to the public or limited to use within an organization for maintaining its internal knowledge base.

Wikis are enabled by wiki software, otherwise known as wiki engines. A wiki engine, being a form of a content management system, differs from other web-based systems such as blog software or static site generators, in that the content is created without any defined owner or leader, and wikis have little inherent structure, allowing structure to emerge according to the needs of the users.[1] Wiki engines usually allow content to be written using a simplified markup language and sometimes edited with the help of a rich-text editor.[2] There are dozens of different wiki engines in use, both standalone and part of other software, such as bug tracking systems. Some wiki engines are free and open-source, whereas others are proprietary. Some permit control over different functions (levels of access); for example, editing rights may permit changing, adding, or removing material. Others may permit access without enforcing access control. Further rules may be imposed to organize content.

There are hundreds of thousands of wikis in use, both public and private, including wikis functioning as knowledge management resources, note-taking tools, community websites, and intranets. Ward Cunningham, the developer of the first wiki software, WikiWikiWeb, originally described wiki as "the simplest online database that could possibly work".[3] "Wiki" (pronounced [wiki][note 1]) is a Hawaiian word meaning "quick".[4][5][6]

The online encyclopedia project Wikipedia is the most popular wiki-based website, as well being one of the most popular websites on the entire internet, having been ranked consistently as such since at least 2007.[7] Wikipedia is not a single wiki but rather a collection of hundreds of wikis, with each one pertaining to a specific language. The English-language Wikipedia has the largest collection of articles, standing at 6,810,885 as of April 2024.[8]

Characteristics

Ward Cunningham
In their 2001 book The Wiki Way: Quick Collaboration on the Web, Ward Cunningham and co-author Bo Leuf described the essence of the Wiki concept:[9][10][page needed]

"A wiki invites all users—not just experts—to edit any page or to create new pages within the wiki web site, using only a standard 'plain-vanilla' Web browser without any extra add-ons."
"Wiki promotes meaningful topic associations between different pages by making page link creation intuitively easy and showing whether an intended target page exists or not."
"A wiki is not a carefully crafted site created by experts and professional writers and designed for casual visitors. Instead, it seeks to involve the typical visitor/user in an ongoing process of creation and collaboration that constantly changes the website landscape."
A wiki enables communities of editors and contributors to write documents collaboratively. All that people require to contribute is a computer, Internet access, a web browser, and a basic understanding of a simple markup language (e.g. MediaWiki markup language). A single page in a wiki website is referred to as a "wiki page", while the entire collection of pages, which are usually well-interconnected by hyperlinks, is "the wiki". A wiki is essentially a database for creating, browsing, and searching through information. A wiki allows non-linear, evolving, complex, and networked text, while also allowing for editor argument, debate, and interaction regarding the content and formatting.[11] A defining characteristic of wiki technology is the ease with which pages can be created and updated. Generally, there is no review by a moderator or gatekeeper before modifications are accepted and thus lead to changes on the website. Many wikis are open to alteration by the general public without requiring registration of user accounts. Many edits can be made in real-time and appear almost instantly online, but this feature facilitates abuse of the system. Private wiki servers require user authentication to edit pages, and sometimes even to read them. Maged N. Kamel Boulos, Cito Maramba, and Steve Wheeler write that the open wikis produce a process of Social Darwinism. "... because of the openness and rapidity that wiki pages can be edited, the pages undergo an evolutionary selection process, not unlike that which nature subjects to living organisms. 'Unfit' sentences and sections are ruthlessly culled, edited and replaced if they are not considered 'fit', which hopefully results in the evolution of a higher quality and more relevant page."[12]

Editing
"Wikitext" redirects here. For the Wikipedia help page, see Help:Wikitext.
Source editing
Some wikis have an edit button or link directly on the page being viewed if the user has permission to edit the page. This can lead to a text-based editing page where participants can structure and format wiki pages with a simplified markup language, sometimes known as wikitext, wiki markup or wikicode (it can also lead to a WYSIWYG editing page; see the paragraph after the table below). For example, starting lines of text with asterisks could create a bulleted list. The style and syntax of wikitexts can vary greatly among wiki implementations,[example needed] some of which also allow HTML tags.

Layout consistency
Wikis have traditionally employed plain-text editing, utilizing simpler conventions than HTML to denote style and structure. Restricting access to HTML and Cascading Style Sheets (CSS) within wikis hinders users from modifying content layout and formatting. However, this restriction offers advantages. It fosters uniformity in appearance by curbing CSS modifications and ensures that users cannot introduce JavaScript code that might impede access for others.

Basic syntax
MediaWiki syntax
(the source code used to add formatting to text)	HTML equivalent
(web code used to add formatting to text)	Rendered output
(as seen by visitors of the wiki)
"Take some more [[tea]]," the March Hare said to Alice, very earnestly.

"I've had '''nothing''' yet," Alice replied in an offended tone, "so I can't take more."

"You mean you can't take ''less''," said the Hatter. "It's very easy to take ''more'' than nothing."
<p>"Take some more <a href="/wiki/Tea" title="Tea">tea</a>," the March Hare said to Alice, very earnestly.</p>

<p>"I've had <b>nothing</b> yet," Alice replied in an offended tone, "so I can't take more."</p>

<p>"You mean you can't take <i>less</i>," said the Hatter. "It's very easy to take <i>more</i> than nothing."</p>

"Take some more tea," the March Hare said to Alice, very earnestly.
"I've had nothing yet," Alice replied in an offended tone, "so I can't take more."

"You mean you can't take less," said the Hatter. "It's very easy to take more than nothing."


Visual editing
Wikis can also make WYSIWYG editing available to users, usually through a JavaScript control that translates graphically entered formatting instructions into the corresponding HTML tags or wikitext. In those implementations, the markup of a newly edited, marked-up version of the page is generated and submitted to the server transparently, shielding the user from this technical detail. An example of this is the VisualEditor on Wikipedia. WYSIWYG controls do not, however, always provide all the features available in wikitext, and some users prefer not to use a WYSIWYG editor. Hence, many of these sites offer some means to edit the wikitext directly.

Version history
Some wikis keep a record of changes made to wiki pages; often, every version of the page is stored. This means that authors can revert to an older version of the page should it be necessary because a mistake has been made, such as the content accidentally being deleted or the page has been vandalized to include offensive or malicious text or other inappropriate content.

Edit summary
"Edit summary" redirects here. For the Wikipedia help page, see Help:Edit summary.
Many wiki implementations, such as MediaWiki, the software that powers Wikipedia, allow users to supply an edit summary when they edit a page. This is a short piece of text summarizing the changes they have made (e.g. "Corrected grammar" or "Fixed formatting in table"). It is not inserted into the article's main text but is stored along with that revision of the page, allowing users to explain what has been done and why. This is similar to a log message when making changes in a revision-control system. This enables other users to see which changes have been made by whom and why, often in a list of summaries, dates and other short, relevant content, a list which is called a "log" or "history".

Navigation
Within the text of most pages, there are usually many hypertext links to other pages within the wiki. This form of non-linear navigation is more "native" to a wiki than structured/formalized navigation schemes. Users can also create any number of index or table-of-contents pages, with hierarchical categorization or whatever form of organization they like. These may be challenging to maintain "by hand", as multiple authors and users may create and delete pages in an ad hoc, unorganized manner. Wikis can provide one or more ways to categorize or tag pages to support the maintenance of such index pages. Some wikis, including the original, have a backlink feature, which displays all pages that link to a given page. It is also typically possible in a wiki to create links to pages that do not yet exist, as a way to invite others to share what they know about a subject new to the wiki. Wiki users can typically "tag" pages with categories or keywords, to make it easier for other users to find the article. For example, a user creating a new article on cold-weather biking might "tag" this page under the categories of commuting, winter sports and bicycling. This would make it easier for other users to find the article.

Linking and creating pages
Links are created using a specific syntax, the so-called "link pattern". Originally, most wikis[citation needed] used CamelCase to name pages and create links. These are produced by capitalizing words in a phrase and removing the spaces between them (the word "CamelCase" is itself an example). While CamelCase makes linking easy, it also leads to links in a form that deviates from the standard spelling. To link to a page with a single-word title, one must abnormally capitalize one of the letters in the word (e.g. "WiKi" instead of "Wiki"). CamelCase-based wikis are instantly recognizable because they have many links with names such as "TableOfContents" and "BeginnerQuestions". A wiki can render the visible anchor of such links "pretty" by reinserting spaces, and possibly also reverting to lower case. This reprocessing of the link to improve the readability of the anchor is, however, limited by the loss of capitalization information caused by CamelCase reversal. For example, "RichardWagner" should be rendered as "Richard Wagner", whereas "PopularMusic" should be rendered as "popular music". There is no easy way to determine which capital letters should remain capitalized. As a result, many wikis now have "free linking" using brackets, and some disable CamelCase by default.

Searching
Most wikis offer at least a title search, and sometimes a full-text search. The scalability of the search depends on whether the wiki engine uses a database. Some wikis, such as PmWiki, use flat files.[13] MediaWiki's first versions used flat files, but it was rewritten by Lee Daniel Crocker in the early 2000s (decade) to be a database application.[citation needed] Indexed database access is necessary for high speed searches on large wikis. Alternatively, external search engines such as Google Search can sometimes be used on wikis with limited searching functions to obtain more precise results.

History
Main article: History of wikis

Wiki Wiki Shuttle at Honolulu International Airport
WikiWikiWeb was the first wiki.[14] Ward Cunningham started developing WikiWikiWeb in Portland, Oregon, in 1994, and installed it on the Internet domain c2.com on March 25, 1995. It was named by Cunningham, who remembered a Honolulu International Airport counter employee telling him to take the "Wiki Wiki Shuttle" bus that runs between the airport's terminals. According to Cunningham, "I chose wiki-wiki as an alliterative substitute for 'quick' and thereby avoided naming this stuff quick-web."[15][16]

Cunningham was, in part, inspired by the Apple HyperCard, which he had used. HyperCard, however, was single-user.[17] Apple had designed a system allowing users to create virtual "card stacks" supporting links among the various cards. Cunningham developed Vannevar Bush's ideas by allowing users to "comment on and change one another's text."[2][18] Cunningham says his goals were to link together people's experiences to create a new literature to document programming patterns, and to harness people's natural desire to talk and tell stories with a technology that would feel comfortable to those not used to "authoring".[17]

Wikipedia became the most famous wiki site, launched in January 2001 and entering the top ten most popular websites in 2007. In the early 2000s (decade), wikis were increasingly adopted in enterprise as collaborative software. Common uses included project communication, intranets, and documentation, initially for technical users. Some companies use wikis as their only collaborative software and as a replacement for static intranets, and some schools and universities use wikis to enhance group learning. There may be greater use of wikis behind firewalls than on the public Internet. On March 15, 2007, the word wiki was listed in the online Oxford English Dictionary.[19]

Alternative definitions
In the late 1990s and early 2000s, the word "wiki" was used to refer to both user-editable websites and the software that powers them; the latter definition is still occasionally in use.[1] Wiki inventor Ward Cunningham wrote in 2014[20] that the word "wiki" should not be used to refer to a single website, but rather to a mass of user-editable pages or sites so that a single website is not "a wiki" but "an instance of wiki". He wrote that the concept of wiki federation, in which the same content can be hosted and edited in more than one location in a manner similar to distributed version control, meant that the concept of a single discrete "wiki" no longer made sense.[21]

Implementations
See also: List of wiki software
Wiki software is a type of collaborative software that runs a wiki system, allowing web pages to be created and edited using a common web browser. It may be implemented as a series of scripts behind an existing web server or as a standalone application server that runs on one or more web servers. The content is stored in a file system, and changes to the content are stored in a relational database management system. A commonly implemented software package is MediaWiki, which runs Wikipedia. Alternatively, personal wikis run as a standalone application on a single computer.

Wikis can also be created on a "wiki farm", where the server-side software is implemented by the wiki farm owner. Some wiki farms can also make private, password-protected wikis. Free wiki farms generally contain advertising on every page. For more information, see Comparison of wiki hosting services.

Trust and security
Controlling changes
"Recent changes" redirects here. For the Wikipedia help page, see Help:Recent changes. For the recent changes page itself, see Special:RecentChanges.

History comparison reports highlight the changes between two revisions of a page.
Wikis are generally designed with the philosophy of making it easy to correct mistakes, rather than making it difficult to make them. Thus, while wikis are very open, they provide a means to verify the validity of recent additions to the body of pages. The most prominent, on almost every wiki, is the "Recent Changes" page—a specific list showing recent edits, or a list of edits made within a given time frame.[22] Some wikis can filter the list to remove minor edits and edits made by automatic importing scripts ("bots").[23] From the change log, other functions are accessible in most wikis: the revision history shows previous page versions and the diff feature highlights the changes between two revisions. Using the revision history, an editor can view and restore a previous version of the article. This gives great power to the author to eliminate edits. The diff feature can be used to decide whether or not this is necessary. A regular wiki user can view the diff of an edit listed on the "Recent Changes" page and, if it is an unacceptable edit, consult the history, restoring a previous revision; this process is more or less streamlined, depending on the wiki software used.[24]

In case unacceptable edits are missed on the "recent changes" page, some wiki engines provide additional content control. It can be monitored to ensure that a page, or a set of pages, keeps its quality. A person willing to maintain pages will be warned of modifications to the pages, allowing them to verify the validity of new editions quickly. This can be seen as a very pro-author and anti-editor feature.[25] A watchlist is a common implementation of this. Some wikis also implement "patrolled revisions", in which editors with the requisite credentials can mark some edits as not vandalism. A "flagged revisions" system can prevent edits from going live until they have been reviewed.[26]

Trustworthiness and reliability of content
Critics of publicly editable wiki systems argue that these systems could be easily tampered with by malicious individuals ("vandals") or even by well-meaning but unskilled users who introduce errors into the content, while proponents maintain that the community of users can catch such malicious or erroneous content and correct it.[2] Lars Aronsson, a data systems specialist, summarizes the controversy as follows: "Most people when they first learn about the wiki concept, assume that a Web site that can be edited by anybody would soon be rendered useless by destructive input. It sounds like offering free spray cans next to a grey concrete wall. The only likely outcome would be ugly graffiti and simple tagging and many artistic efforts would not be long lived. Still, it seems to work very well."[14] High editorial standards in medicine and health sciences articles, in which users typically use peer-reviewed journals or university textbooks as sources, have led to the idea of expert-moderated wikis.[27] Some wikis allow one to link to specific versions of articles, which has been useful to the scientific community, in that expert peer reviewers could analyse articles, improve them and provide links to the trusted version of that article.[28] Noveck points out that "participants are accredited by members of the wiki community, who have a vested interest in preserving the quality of the work product, on the basis of their ongoing participation." On controversial topics that have been subject to disruptive editing, a wiki author may restrict editing to registered users.[29]

Security
"Edit war" redirects here. Not to be confused with Edit conflict.
For Wikipedia's policy on edit warring, see Wikipedia:Edit warring.
The open philosophy of wiki – allowing anyone to edit content – does not ensure that every editor's intentions are well-mannered. For example, vandalism (changing wiki content to something offensive, adding nonsense, maliciously removing content, or deliberately adding incorrect information, such as hoax information) can be a major problem. On larger wiki sites, such as those run by the Wikimedia Foundation, vandalism can go unnoticed for some period of time. Wikis, because of their open nature, are susceptible to intentional disruption, known as "trolling". Wikis tend to take a soft-security approach to the problem of vandalism, making damage easy to undo rather than attempting to prevent damage. Larger wikis often employ sophisticated methods, such as bots that automatically identify and revert vandalism and JavaScript enhancements that show characters that have been added in each edit. In this way, vandalism can be limited to just "minor vandalism" or "sneaky vandalism", where the characters added/eliminated are so few that bots do not identify them and users do not pay much attention to them.[30][unreliable source] An example of a bot that reverts vandalism on Wikipedia is ClueBot NG. ClueBot NG, which uses machine learning to identify likely vandalism, can revert edits, often within minutes, if not seconds.[31]

The amount of vandalism a wiki receives depends on how open the wiki is. For instance, some wikis allow unregistered users, identified by their IP addresses, to edit content, while others limit this function to just registered users.[32]

Edit wars can also occur as users repetitively revert a page to the version they favor. In some cases, editors with opposing views of which content should appear or what formatting style should be used will change and re-change each other's edits. This results in the page being "unstable" from a general user's perspective, because each time a general user comes to the page, it may look different. Some wiki software allows an administrator to stop such edit wars by locking a page from further editing until a decision has been made on what version of the page would be most appropriate.[11] Some wikis are in a better position than others to control behavior due to governance structures existing outside the wiki. For instance, a college teacher can create incentives for students to behave themselves on a class wiki they administer by limiting editing to logged-in users and pointing out that all contributions can be traced back to the contributors. Bad behavior can then be dealt with under university policies.[13]

Potential malware vector
Malware can also be a problem for wikis, as users can add links to sites hosting malicious code. For example, a German Wikipedia article about the Blaster Worm was edited to include a hyperlink to a malicious website. Users of vulnerable Microsoft Windows systems who followed the link would be infected.[11] A countermeasure is the use of software that prevents users from saving an edit that contains a link to a site listed on a blacklist of malicious sites.

Communities
Applications

The home page of the English Wikipedia
The English Wikipedia has the largest user base among wikis on the World Wide Web[33] and ranks in the top 10 among all Web sites in terms of traffic.[34] Other large wikis include the WikiWikiWeb, Memory Alpha, Wikivoyage, and previously Susning.nu, a Swedish-language knowledge base. Medical and health-related wiki examples include Ganfyd, an online collaborative medical reference that is edited by medical professionals and invited non-medical experts.[12] Many wiki communities are private, particularly within enterprises. They are often used as internal documentation for in-house systems and applications. Some companies use wikis to allow customers to help produce software documentation.[35] A study of corporate wiki users found that they could be divided into "synthesizers" and "adders" of content. Synthesizers' frequency of contribution was affected more by their impact on other wiki users, while adders' contribution frequency was affected more by being able to accomplish their immediate work.[36] From a study of thousands of wiki deployments, Jonathan Grudin concluded careful stakeholder analysis and education are crucial to successful wiki deployment.[37]

In 2005, the Gartner Group, noting the increasing popularity of wikis, estimated that they would become mainstream collaboration tools in at least 50% of companies by 2009.[38][needs update] Wikis can be used for project management.[39][40][unreliable source] Wikis have also been used in the academic community for sharing and dissemination of information across institutional and international boundaries.[41] In those settings, they have been found useful for collaboration on grant writing, strategic planning, departmental documentation, and committee work.[42] In the mid-2000s, the increasing trend among industries toward collaboration placed a heavier impetus upon educators to make students proficient in collaborative work, inspiring even greater interest in wikis being used in the classroom.[11]

Wikis have found some use within the legal profession and within the government. Examples include the Central Intelligence Agency's Intellipedia, designed to share and collect intelligence, DKospedia, which was used by the American Civil Liberties Union to assist with review of documents about the internment of detainees in Guantánamo Bay;[43] and the wiki of the United States Court of Appeals for the Seventh Circuit, used to post court rules and allow practitioners to comment and ask questions. The United States Patent and Trademark Office operates Peer-to-Patent, a wiki to allow the public to collaborate on finding prior art relevant to the examination of pending patent applications. Queens, New York has used a wiki to allow citizens to collaborate on the design and planning of a local park. Cornell Law School founded a wiki-based legal dictionary called Wex, whose growth has been hampered by restrictions on who can edit.[29]

In academic contexts, wikis have also been used as project collaboration and research support systems.[44][45]

City wikis
A city wiki (or local wiki) is a wiki used as a knowledge base and social network for a specific geographical locale.[46][47][48] The term 'city wiki' or its foreign language equivalent (e.g. German 'Stadtwiki') is sometimes also used for wikis that cover not just a city, but a small town or an entire region. A city wiki contains information about specific instances of things, ideas, people and places. Much of this information might not be appropriate for encyclopedias such as Wikipedia (e.g. articles on every retail outlet in a town), but might be appropriate for a wiki with more localized content and viewers. A city wiki could also contain information about the following subjects, that may or may not be appropriate for a general knowledge wiki, such as:

Details of public establishments such as public houses, bars, accommodation or social centers
Owner name, opening hours and statistics for a specific shop
Statistical information about a specific road in a city
Flavors of ice cream served at a local ice cream parlor
A biography of a local mayor and other persons
WikiNodes
"WikiNode" redirects here. For the app for the Apple iPad, see WikiNodes.
Visualization of the collaborative work in the German wiki project Mathe für Nicht-Freaks
WikiNodes are pages on wikis that describe related wikis. They are usually organized as neighbors and delegates. A neighbor wiki is simply a wiki that may discuss similar content or may otherwise be of interest. A delegate wiki is a wiki that agrees to have certain content delegated to that wiki.[49] One way of finding a wiki on a specific subject is to follow the wiki-node network from wiki to wiki.

Participants
The four basic types of users who participate in wikis are reader, author, wiki administrator and system administrator. The system administrator is responsible for the installation and maintenance of the wiki engine and the container web server. The wiki administrator maintains wiki content and is provided additional functions about pages (e.g. page protection and deletion), and can adjust users' access rights by, for instance, blocking them from editing.[50]

Growth factors
A study of several hundred wikis showed that a relatively high number of administrators for a given content size is likely to reduce growth;[51] that access controls restricting editing to registered users tends to reduce growth; that a lack of such access controls tends to fuel new user registration; and that higher administration ratios (i.e. admins/user) have no significant effect on content or population growth.[52]

Conferences
Active conferences and meetings about wiki-related topics include:

Atlassian Summit, an annual conference for users of Atlassian software, including Confluence.[53]
OpenSym (called WikiSym until 2014), an academic conference dedicated to research about wikis and open collaboration.
SMWCon, a bi-annual conference for users and developers of Semantic MediaWiki.[54]
TikiFest, a frequently held meeting for users and developers of Tiki Wiki CMS Groupware.[55]
Wikimania, an annual conference dedicated to the research and practice of Wikimedia Foundation projects like Wikipedia.
Former wiki-related events include:

RecentChangesCamp (2006–2012), an unconference on wiki-related topics.
RegioWikiCamp (2009–2013), a semi-annual unconference on "regiowikis", or wikis on cities and other geographic areas.[56]
Legal environment
Joint authorship of articles, in which different users participate in correcting, editing, and compiling the finished product, can also cause editors to become tenants in common of the copyright, making it impossible to republish without permission of all co-owners, some of whose identities may be unknown due to pseudonymous or anonymous editing.[11] Where persons contribute to a collective work such as an encyclopedia, there is, however, no joint ownership if the contributions are separate and distinguishable.[57] Despite most wikis' tracking of individual contributions, the action of contributing to a wiki page is still arguably one of jointly correcting, editing, or compiling, which would give rise to joint ownership. Some copyright issues can be alleviated through the use of an open content license. Version 2 of the GNU Free Documentation License includes a specific provision for wiki relicensing; Creative Commons licenses are also popular. When no license is specified, an implied license to read and add content to a wiki may be deemed to exist on the grounds of business necessity and the inherent nature of a wiki, although the legal basis for such an implied license may not exist in all circumstances.[citation needed]

Wikis and their users can be held liable for certain activities that occur on the wiki. If a wiki owner displays indifference and forgoes controls (such as banning copyright infringers) that they could have exercised to stop copyright infringement, they may be deemed to have authorized infringement, especially if the wiki is primarily used to infringe copyrights or obtains a direct financial benefit, such as advertising revenue, from infringing activities.[11] In the United States, wikis may benefit from Section 230 of the Communications Decency Act, which protects sites that engage in "Good Samaritan" policing of harmful material, with no requirement on the quality or quantity of such self-policing.[58] It has also been argued, however, that a wiki's enforcement of certain rules, such as anti-bias, verifiability, reliable sourcing, and no-original-research policies, could pose legal risks.[59] When defamation occurs on a wiki, theoretically, all users of the wiki can be held liable, because any of them had the ability to remove or amend the defamatory material from the "publication". It remains to be seen whether wikis will be regarded as more akin to an internet service provider, which is generally not held liable due to its lack of control over publications' contents, than a publisher.[11] It has been recommended that trademark owners monitor what information is presented about their trademarks on wikis, since courts may use such content as evidence pertaining to public perceptions. Joshua Jarvis notes, "Once misinformation is identified, the trademark owner can simply edit the entry".[60]

See also
icon	Internet portal
Comparison of wiki software
Content management system
CURIE
Dispersed knowledge
List of wikis
Mass collaboration
Universal Edit Button
Wikis and education
Notes
 The realization of the Hawaiian /w/ phoneme varies between [w] and [v], and the realization of the /k/ phoneme varies between [k] and [t], among other realizations. Thus, the pronunciation of the Hawaiian word wiki varies between ['wiki], ['witi], ['viki], and ['viti]. See Hawaiian phonology for more details.
References
 Mitchell, Scott (July 2008), Easy Wiki Hosting, Scott Hanselman's blog, and Snagging Screens, MSDN Magazine, archived from the original on March 16, 2010, retrieved March 9, 2010
 "wiki", Encyclopædia Britannica, vol. 1, London: Encyclopædia Britannica, Inc., 2007, archived from the original on April 24, 2008, retrieved April 10, 2008
 Cunningham, Ward (June 27, 2002). "What is a Wiki". WikiWikiWeb. Archived from the original on April 16, 2008. Retrieved April 10, 2008.
 "Hawaiian Words; Hawaiian to English". mauimapp.com. Archived from the original on September 14, 2008. Retrieved September 19, 2008.
 Hasan, Heather (2012), Wikipedia, 3.5 million articles and counting, New York : Rosen Central, p. 11, ISBN 9781448855575, archived from the original on October 26, 2019, retrieved August 6, 2019
 Andrews, Lorrin (1865), A dictionary of the Hawaiian language to which is appended an English-Hawaiian vocabulary and a chronological table of remarkable events, Henry M. Whitney, p. 514, archived from the original on August 15, 2014, retrieved June 1, 2014
 "Alexa Top Sites". Archived from the original on March 2, 2015. Retrieved December 1, 2016.
 "Wikipedia:Size of Wikipedia". Wikipedia. Wikimedia Foundation. Retrieved January 14, 2024.
 Cunningham, Ward; Leuf, Bo (April 13, 2001). The Wiki Way: Quick Collaboration on the Web. Addison–Wesley. ISBN 9780201714999. OCLC 45715320. Google Books page Archived January 11, 2023, at the Wayback Machine
 "Wiki Design Principles". Archived from the original on April 30, 2002. Retrieved April 30, 2002.
 Black, Peter; Delaney, Hayden; Fitzgerald, Brian (2007), Legal Issues for Wikis: The Challenge of User-generated and Peer-produced Knowledge, Content and Culture (PDF), vol. 14, eLaw J., archived from the original (PDF) on December 22, 2012
 Boulos, M. N. K.; Maramba, I.; Wheeler, S. (2006), "Wikis, blogs and podcasts: a new generation of Web-based tools for virtual collaborative clinical practice and education", BMC Medical Education, 6: 41, doi:10.1186/1472-6920-6-41, PMC 1564136, PMID 16911779
 Naomi, Augar; Raitman, Ruth; Zhou, Wanlei (2004). "Teaching and learning online with wikis". Proceedings of Beyond the Comfort Zone: 21st ASCILITE Conference: 95–104. CiteSeerX 10.1.1.133.1456.
 Ebersbach 2008, p. 10
 Cunningham, Ward (November 1, 2003). "Correspondence on the Etymology of Wiki". WikiWikiWeb. Archived from the original on March 17, 2007. Retrieved March 9, 2007.
 Cunningham, Ward (February 25, 2008). "Wiki History". WikiWikiWeb. Archived from the original on June 21, 2002. Retrieved March 9, 2007.
 Bill Venners (October 20, 2003). "Exploring with Wiki: A Conversation with Ward Cunningham, Part I". artima developer. Archived from the original on February 5, 2015. Retrieved December 12, 2014.
 Cunningham, Ward (July 26, 2007). "Wiki Wiki Hyper Card". WikiWikiWeb. Archived from the original on April 6, 2007. Retrieved March 9, 2007.
 Diamond, Graeme (March 1, 2007). "March 2007 update". Oxford English Dictionary. Archived from the original on January 7, 2011. Retrieved March 16, 2007.
 Ward Cunningham [@WardCunningham] (November 8, 2014). "The plural of wiki is wiki. See https://forage.ward.fed.wiki.org/an-install-of-wiki.html" (Tweet). Retrieved March 18, 2019 – via Twitter.
 "Smallest Federated Wiki". wiki.org. Archived from the original on September 28, 2015. Retrieved September 28, 2015.
 Ebersbach 2008, p. 20
 Ebersbach 2008, p. 54
 Ebersbach 2008, p. 178
 Ebersbach 2008, p. 109
 Goldman, Eric, "Wikipedia's Labor Squeeze and its Consequences", Journal on Telecommunications and High Technology Law, 8
 Barsky, Eugene; Giustini, Dean (December 2007). "Introducing Web 2.0: wikis for health librarians" (PDF). Journal of the Canadian Health Libraries Association. 28 (4): 147–150. doi:10.5596/c07-036. ISSN 1708-6892. Archived (PDF) from the original on April 30, 2012. Retrieved November 7, 2011.
 Yager, Kevin (March 16, 2006). "Wiki ware could harness the Internet for science". Nature. 440 (7082): 278. Bibcode:2006Natur.440..278Y. doi:10.1038/440278a. PMID 16541049.
 Noveck, Beth Simone (March 2007), "Wikipedia and the Future of Legal Education", Journal of Legal Education, 57 (1), archived from the original on July 3, 2014(subscription required)
 "Security". Assothink. Archived from the original on January 6, 2014. Retrieved February 16, 2013.
 Hicks, Jesse (February 18, 2014). "This machine kills trolls". The Verge. Archived from the original on August 27, 2014. Retrieved September 7, 2014.
 Ebersbach 2008, p. 108
 "List of largest (Media)wikis". S23-Wiki. April 3, 2008. Archived from the original on August 25, 2014. Retrieved December 12, 2014.
 "Alexa Top 500 Global Sites". Alexa Internet. Archived from the original on March 2, 2015. Retrieved April 26, 2015.
 Müller, C.; Birn, L. (September 6–8, 2006). "Wikis for Collaborative Software Documentation" (PDF). i-know.tugraz.at. Proceedings of I-KNOW '06. Archived from the original (PDF) on July 6, 2011.
 Majchrzak, A.; Wagner, C.; Yates, D. (2006), "Corporate wiki users: results of a survey", Proceedings of the 2006 international symposium on Wikis, Symposium on Wikis, pp. 99–104, doi:10.1145/1149453.1149472, ISBN 978-1-59593-413-0, S2CID 13206858
 Grudin, Jonathan; Poole, Erika Shehan (2015). "Wikis at work: Success factors and challenges for sustainability of enterprise wikis". Microsoft Research. Archived from the original on September 4, 2015. Retrieved June 16, 2015.
 Conlin, Michelle (November 28, 2005), "E-Mail Is So Five Minutes Ago", Bloomberg BusinessWeek, archived from the original on October 17, 2012
 "HomePage". Project Management Wiki.org. Archived from the original on August 16, 2014. Retrieved May 8, 2012.
 "Ways to Wiki: Project Management". EditMe. January 4, 2010. Archived from the original on May 8, 2012.
 Wanderley, M. M.; Birnbaum, D.; Malloch, J. (2006). "SensorWiki.org: a collaborative resource for researchers and interface designers". NIME '06 Proceedings of the 2006 Conference on New Interfaces for Musical Expression. IRCAM – Centre Pompidou: 180–183. ISBN 978-2-84426-314-8.
 Lombardo, Nancy T. (June 2008). "Putting Wikis to Work in Libraries". Medical Reference Services Quarterly. 27 (2): 129–145. doi:10.1080/02763860802114223. PMID 18844087. S2CID 11552140.
 Noveck, Beth Simone (2007). "Wikipedia and the Future of Legal Education". Journal of Legal Education. 57: 3.
 Au, C. H. (December 2017). "Wiki as a research support system — A trial in information systems research". 2017 IEEE International Conference on Industrial Engineering and Engineering Management (IEEM). pp. 2271–2275. doi:10.1109/IEEM.2017.8290296. ISBN 978-1-5386-0948-4. S2CID 44029462.
 Au, Cheuk-hang. "Using Wiki for Project Collaboration – with Comparison on Facebook" (PDF). Archived (PDF) from the original on April 12, 2019.
 Andersen, Michael (November 6, 2009). "Welcome to Davis, Calif.: Six lessons from the world's best local wiki". Nieman Lab. Archived from the original on August 8, 2013. Retrieved January 6, 2023.
 McGann, Laura (June 18, 2010). "Knight News Challenge: Is a wiki site coming to your city? Local Wiki will build software to make it simple". Nieman Lab. Archived from the original on June 25, 2013. Retrieved January 6, 2023.
 Wired: Makice, Kevin (July 15, 2009). Hey, Kid: Support Your Local Wiki Archived April 27, 2015, at the Wayback Machine
 "Frequently Asked Questions". WikiNodes. Archived from the original on August 10, 2007.
 Cubric, Marija (2007). "Analysis of the use of Wiki-based collaborations in enhancing student learning". UH Business School Working Paper. University of Hertfordshire. Archived from the original on May 15, 2011. Retrieved April 25, 2011.
 Roth, C.; Taraborelli, D.; Gilbert, N. (2008). "Measuring wiki viability. An empirical assessment of the social dynamics of a large sample of wikis" (PDF). nitens.org. The Centre for Research in Social Simulation: 3. Archived (PDF) from the original on October 11, 2017. Figure 4 shows that having a relatively high number of administrators for a given content size is likely to reduce growth.
 Roth, C.; Taraborelli, D.; Gilbert, N. (2008). "Measuring wiki viability. An empirical assessment of the social dynamics of a large sample of wikis" (PDF). Surrey Research Insight Open Access. The Centre for Research in Social Simulation. Archived from the original (PDF) on June 16, 2012. Retrieved November 9, 2018.
 "Atlassian". Summit.atlassian.com. Archived from the original on June 13, 2011. Retrieved June 20, 2011.
 "SMWCon". Semantic-mediawiki.org. Archived from the original on July 14, 2011. Retrieved June 20, 2011.
 "TikiFest". Tiki.org. Archived from the original on June 30, 2011. Retrieved June 20, 2011.
 "Regiowiki Main Page". Wiki.regiowiki.eu. Archived from the original on August 13, 2009. Retrieved June 20, 2011.
 "Redwood Music Ltd v. B Feldman & Co Ltd and others". Reports of Patent, Design and Trade Mark Cases. Oxford University Press. September 1979. doi:10.1093/rpc/1979rpc385. Archived from the original on October 17, 2022.
 Walsh, Kathleen M.; Oh, Sarah (February 23, 2010). "Self-Regulation: How Wikipedia Leverages User-Generated Quality Control Under Section 230". Archived from the original on January 6, 2014.
 Myers, Ken S. (2008), "Wikimmunity: Fitting the Communications Decency Act to Wikipedia", Harvard Journal of Law and Technology, 20, The Berkman Center for Internet and Society: 163, SSRN 916529, archived from the original on January 24, 2024
 Jarvis, Joshua (May 2008), "Police your marks in a wiki world", Managing Intellectual Property, 179 (179): 101–103, archived from the original on March 4, 2016
Sources
Ebersbach, Anja (2008), Wiki: Web Collaboration, Springer Science+Business Media, ISBN 978-3-540-35150-4
Further reading
Mader, Stewart (December 10, 2007), Wikipatterns, John Wiley & Sons, ISBN 978-0-470-22362-8
Tapscott, Don (April 17, 2008), Wikinomics: How Mass Collaboration Changes Everything, Portfolio Hardcover, ISBN 978-1-59184-193-7
External links
Wiki
at Wikipedia's sister projects
Definitions from Wiktionary
Media from Commons
News from Wikinews
Resources from Wikiversity
Data from Wikidata
Documentation from MediaWiki
Listen to this article (16 minutes)
Duration: 15 minutes and 36 seconds.15:36Spoken Wikipedia icon
This audio file was created from a revision of this article dated 14 March 2007, and does not reflect subsequent edits.
(Audio help · More spoken articles)
Wiki at Curlie
Exploring with Wiki, an interview with Ward Cunningham by Bill Verners
Murphy, Paula (April 2006). Topsy-turvy World of Wiki. University of California.
Ward Cunningham's correspondence with etymologists
WikiIndex and WikiApiary, directories of wikis
WikiMatrix, a website for comparing wiki software and hosts
wikiteam on GitHub
vte
Wikis
Types	
PersonalMedicalSemantic
Components	
Software
Lists	
List of LocalWikisWikisSoftwareList of WikipediasList of Wiktionaries
Comparisons	
SoftwareWiki farms
Notable wikis	
BallotpediaBiographiconBook DrumChalo ChatuConservapediaDavisWikiDiplopediaEncyclopedia DramaticaEngineering and Technology History WikiFamily History Research WikiGene WikiGeo-WikiGiant BombGynopediaThe Hidden WikiIntellipediaLocalWikiNamuwikiOpen protein structure annotation networkRationalWikiResistance ManualRigveda WikiSky-Map.orgThe Cutting Room FloorTV TropesUncyclopediaWikiArtWikiFactorWikifoniawikiHowWikilocWikimaniaWikipediaWikiProfessionalWikiprogressWikiratingWikiStageWikistratWikiTribuneWowpedia
Wiki farms	
ConfluenceFandomPBworksWetpaint
See also	
Wikis and educationHistoryCreole.wiki
vte
Wiki software
vte
Computer-mediated communication
vte
Sharing economy
Authority control databases Edit this at Wikidata
Categories: WikisHawaiian words and phrasesHypertextSelf-organizationSocial information processing
This page was last edited on 11 April 2024, at 20:11 (UTC).
Text is available under the Creative Commons Attribution-ShareAlike License 4.0; additional terms may apply. By using this site, you agree to the Terms of Use and Privacy Policy. Wikipedia® is a registered trademark of the Wikimedia Foundation, Inc., a non-profit organization.
Privacy policyAbout WikipediaDisclaimersContact WikipediaCode of ConductDevelopersStatisticsCookie statementMobile viewWikimedia FoundationPowered by MediaWiki
Toggle limited content width


"""

In [17]:
from Tokenizer_help_funtions import pair_switch , get_pair_counts

In [101]:
class gpt4_tokenizer:
    def __init__(self,text,vocab_size):
        self.text=text
        self.vocab_size=vocab_size
        self.merges={}
        self.vocab = {idx: bytes([idx]) for idx in range(256)}

    def train(self, text,vocab_size, verbose=False):
        text = text.encode("utf-8")
        text=list(map(int,text))

        number_of_merges=vocab_size-256
        bpe_tokens=list(text)

        for i in range(number_of_merges):
            pair_dict = get_pair_counts(bpe_tokens)
            pair = max(pair_dict, key=pair_dict.get)
            if verbose:
                print(f"Token {pair} is merged as {i+256}")
            bpe_tokens = pair_switch(bpe_tokens, pair, i+256)
            self.merges[pair] = i+256
        return self.merges, bpe_tokens
    
    def decode(self,ids):

    
        for (p0, p1), idx in merges.items():
            self.vocab[idx] = self.vocab[p0] + self.vocab[p1]

        tokens = b"".join(self.vocab[idx] for idx in ids)
        decoded_text = tokens.decode("utf-8", errors="replace")
        return decoded_text
    
    def encoder(self,text):
        # Convert the text to bytes
        tokens=list(text.encode("utf-8"))
        tokens=list(map(int,tokens))
        while len(tokens)>=2:
            pair_counts=get_pair_counts(tokens)
            pair=min(pair_counts,key=lambda p:self.merges.get(p,float("inf")))
            if pair not in self.merges:
                break
            idx = self.merges[pair]
            tokens=pair_switch(tokens,pair,idx)
        return tokens
        

In [102]:
tokenizer = gpt4_tokenizer(text=text, vocab_size=12000)

In [103]:
merges, bpe_tokens = tokenizer.train(text=text,vocab_size=12000, verbose=True)

Token (101, 32) is merged as 256
Token (115, 32) is merged as 257
Token (105, 110) is merged as 258
Token (116, 104) is merged as 259
Token (101, 114) is merged as 260
Token (44, 32) is merged as 261
Token (111, 110) is merged as 262
Token (100, 32) is merged as 263
Token (97, 110) is merged as 264
Token (111, 114) is merged as 265
Token (116, 32) is merged as 266
Token (105, 107) is merged as 267
Token (267, 105) is merged as 268
Token (101, 110) is merged as 269
Token (97, 114) is merged as 270
Token (46, 32) is merged as 271
Token (116, 105) is merged as 272
Token (97, 108) is merged as 273
Token (121, 32) is merged as 274
Token (259, 256) is merged as 275
Token (101, 263) is merged as 276
Token (258, 103) is merged as 277
Token (114, 101) is merged as 278
Token (111, 32) is merged as 279
Token (111, 102) is merged as 280
Token (100, 105) is merged as 281
Token (97, 32) is merged as 282
Token (99, 104) is merged as 283
Token (119, 268) is merged as 284
Token (262, 32) is merged as 2

In [104]:
tokenizer.decode([256])

'e '

In [106]:
tokenizer.encoder("text is best")

[753, 315, 98, 2054]